In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from IPython import display as ds
import matplotlib.image as mpimg
import cv2
import tensorflow as tf
import math

%matplotlib inline

/home/kvpcloud/.conda/envs/p3_gpu/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
class Dataset:
    main = 'dataset/UCSD_Anomaly_Dataset.v1p2'
    train_images = np.load('{}/train.npy'.format(main))
    train_images_ = np.load('{}/train_.npy'.format(main))
    test_images = np.load('{}/test.npy'.format(main))
    test_images_ = np.load('{}/test_.npy'.format(main))

In [3]:
Dataset.train_images.shape

(200, 158, 238, 3)

In [4]:
Dataset.train_images[0]

array([[[0.36078431, 0.36078431, 0.36078431],
        [0.35686275, 0.35686275, 0.35686275],
        [0.31764706, 0.31764706, 0.31764706],
        ...,
        [0.16470588, 0.16470588, 0.16470588],
        [0.22352941, 0.22352941, 0.22352941],
        [0.29019608, 0.29019608, 0.29019608]],

       [[0.2745098 , 0.2745098 , 0.2745098 ],
        [0.2627451 , 0.2627451 , 0.2627451 ],
        [0.27843137, 0.27843137, 0.27843137],
        ...,
        [0.18823529, 0.18823529, 0.18823529],
        [0.19215686, 0.19215686, 0.19215686],
        [0.33333333, 0.33333333, 0.33333333]],

       [[0.30588235, 0.30588235, 0.30588235],
        [0.32941176, 0.32941176, 0.32941176],
        [0.3372549 , 0.3372549 , 0.3372549 ],
        ...,
        [0.24705882, 0.24705882, 0.24705882],
        [0.16862745, 0.16862745, 0.16862745],
        [0.28235294, 0.28235294, 0.28235294]],

       ...,

       [[0.41960784, 0.41960784, 0.41960784],
        [0.41960784, 0.41960784, 0.41960784],
        [0.43137255, 0

In [5]:
class Params:
    latent_feature_count = [10,10,3]
    epochs = 10
    reduced_feature_rbf_count = 3 # liklihood fails for 3,4&5 and gets stuck for 2
    frames_for_anomaly = 10

In [6]:
class Network:
    
    def __init__(self):
        self.graph = tf.Graph()
        with self.graph.as_default():
            with tf.name_scope('Input'):
                self.x = tf.placeholder(tf.float32, shape=(None, 158, 238, 3), name='X')
                self.x_ = tf.placeholder(tf.float32, shape=(None, 158, 238, 3), name='X_')

        self.encoder, self.decoder = self.build_network(self.x)

    def build_network(self, x):
        def encoder(x):
            with self.graph.as_default():
                with tf.name_scope('Encoder'):
                    x = tf.layers.conv2d(x, filters=32, kernel_size=(5,5), strides=(1,1), activation=tf.nn.relu)
                    print('After 1st Conv', x.get_shape())
                    tf.summary.image('encoder_hidden_1', x[:,:,:,0:3],max_outputs=1)
                    
                    x = tf.layers.max_pooling2d(x, pool_size=(5,5), strides=(1,1), )
                    print('After 1st Pooling', x.get_shape())

                    x = tf.layers.conv2d(x, filters=16, kernel_size=(5,5), strides=(1,1), activation=tf.nn.relu)
                    print('After 2nd Conv', x.get_shape())
                    tf.summary.image('encoder_hidden_2', x[:,:,:,0:3],max_outputs=1)
                    
                    x = tf.layers.max_pooling2d(x, pool_size=(5,5), strides=(1,1))
                    print('After 2nd Pooling', x.get_shape())

                    x = tf.layers.conv2d(x, filters=8, kernel_size=(5,5), strides=(1,1), activation=tf.nn.relu)
                    print('After 3rd Conv', x.get_shape())
                    tf.summary.image('encoder_hidden_3', x[:,:,:,0:3],max_outputs=1)
                    
                    encoded = tf.layers.max_pooling2d(x, pool_size=(5,5), strides=(1,1))
                    print('After 3rd Pooling (Final Encoded)', x.get_shape())
                    tf.summary.image('encoder_hidden_final', encoded[:,:,:,0:3], max_outputs=1)
                    
                    with tf.name_scope('Latent'):
                    
                        self.latent = tf.layers.dense(tf.contrib.layers.flatten(encoded), #depricated
                                                  units=np.prod(Params.latent_feature_count), 
                                                  activation=tf.nn.relu)

                        print('Latent', self.latent.get_shape())
                        tf.summary.histogram('Latent', self.latent)

                
            return encoded

        def decoder(encoded):
            with self.graph.as_default():
                with tf.name_scope('Decoder'):
                    #x = tf.reshape(encoded, [-1] + Params.latent_feature_count)
                    x = encoded
                    x = tf.layers.conv2d_transpose(x, filters=8, kernel_size=(5,5), strides=(1,1), activation=tf.nn.relu)
                    print('After 1st conv transpose', x.get_shape())
                    tf.summary.image('decoder_hidden_1', x[:,:,:,0:3],max_outputs=1)
                    
                    x = tf.layers.conv2d_transpose(x, filters=16, kernel_size=(5,5), strides=(1,1), activation=tf.nn.relu)
                    print('After 2nd conv transpose', x.get_shape())
                    tf.summary.image('decoder_hidden_2', x[:,:,:,0:3],max_outputs=1)
                    
                    x = tf.layers.conv2d_transpose(x, filters=32, kernel_size=(5,5), strides=(1,1), activation=tf.nn.relu)
                    print('After 3rd conv transpose', x.get_shape())
                    tf.summary.image('decoder_hidden_3', x[:,:,:,0:3],max_outputs=1)
                    
                    x = tf.layers.conv2d_transpose(x, filters=32, kernel_size=(5,5), strides=(1,1), activation=tf.nn.relu)
                    print('After 4th conv transpose', x.get_shape())
                    tf.summary.image('decoder_hidden_4', x[:,:,:,0:3],max_outputs=1)
                    
                    x = tf.layers.conv2d_transpose(x, filters=3, kernel_size=(5,5), strides=(1,1), activation=tf.nn.relu)
                    print('After 5th conv transpose', x.get_shape())
                    tf.summary.image('decoder_hidden_5', x[:,:,:,0:3],max_outputs=1)
                    
                    decoded = tf.layers.conv2d_transpose(x, filters=3, kernel_size=(5,5), strides=(1,1), activation=tf.nn.sigmoid)
                    print('After 6th conv transpose (final decoded)', decoded.get_shape())
                    tf.summary.image('decoder_hidden_decoded', decoded[:,:,:,0:3],max_outputs=1)

            return decoded
        
        return encoder, decoder
    
    def get_encoded(self):
        with self.graph.as_default():
            encoded = self.encoder(self.x_)

        return encoded
    
    def get_decoded(self):
        encoded = self.get_encoded()
        x_hat = self.decoder(encoded)
        return x_hat
    
    def lstm(self, latent):
        with self.graph.as_default():

            with tf.name_scope('LSTM') as lstm_scope:
                #x = tf.layers.dense(latent, units=np.prod(Params.latent_feature_count), activation=tf.nn.relu)

                x = latent

                lstm = tf.contrib.rnn.BasicLSTMCell(64)

                # Grouping 10 images into 1 to form a video clip for which Anomaly detection will be done
                x_list = tf.split(x, 10, axis=0) 

                y_list, self.state = tf.nn.static_rnn(lstm, x_list, dtype = tf.float32)

                lstm_out = tf.stack(y_list[-1]) # we only need the last output
                lstm_out = tf.layers.dense(lstm_out, Params.reduced_feature_rbf_count)
                print('Reduced dimensions for RBF', lstm_out.get_shape())
                lstm_out = tf.print(lstm_out, [lstm_out], "********* Output of LSTM, Input to RBF ", summarize=30)
                
        return lstm_out

    def rbf(self, lstm_output):
        with self.graph.as_default():
            def get_cost(U, Z, Q): 

                cost = - (-U - tf.log(Z)) 
                return tf.reduce_mean(cost)

            f = Params.reduced_feature_rbf_count #np.prod(Params.latent_feature_count)


            with tf.name_scope('RBF'):
                X = lstm_output

                mu = tf.Variable(tf.random_uniform([1,f], minval=0.1, dtype=tf.float32))
                mu = tf.print(mu, [mu], "************ mu")
                #Q_ = tf.Variable(tf.truncated_normal([f], mean = 1)) 
                global_step = tf.Variable(0, trainable=False)

                sd = tf.Variable(tf.random_uniform([f], minval=0.1, dtype=tf.float32))

                sigma = tf.square(sd)
                sigma = tf.print(sigma, [sigma], "************ Sigma (squared) ")
                sigma_inverse = tf.reciprocal(sigma)
                sigma_inverse = tf.print(sigma_inverse, [sigma_inverse], '************* sigma_inverse')

                cov_inverse = tf.diag(sigma_inverse)

                det_sigma = tf.reduce_prod(sigma) #tf.pow(sigma, 0.5))
                
                z = tf.multiply(2*math.pi, det_sigma)
                z = tf.print(z, [z], ' ******* z')

                with tf.name_scope('Likelihood'):

                    M = X - mu
                    
                    energy = tf.matmul(tf.matmul(M,cov_inverse), tf.transpose(M)) 
                    #print("Energy after matmul", energy.get_shape())
                    #energy = tf.print(energy, [energy], '*********** Energy after matmul')

                    energy = tf.reduce_sum(energy, axis = 1, keepdims = True)
                    energy = tf.print(energy, [energy], '*********** Energy after summation')
                    energy = -1 * tf.multiply(energy, 0.5)
                    print('Energy after summation (always negative)', energy.get_shape())

                    print('X', X.get_shape())
                    print('Covariance', cov_inverse.get_shape())

                    expnt = tf.exp(energy)
                    print('exponent', expnt.get_shape())
                    expnt = tf.print(expnt, [expnt], "********* Exponent")

                    Y_ = tf.divide(expnt, z)


                    #Y_ = tf.layers.dense(Y_, 1)
                    rbf_out = Y_ #tf.reduce_mean(Y_) # tf.layers.dense(Y_, 1)
                    Y_ = tf.print(Y_, [Y_], 'XXXXXXXXXXXXXXXXXXXX Liklihood XXXXXXXXXXXXXXXXXXXXXXXXXXXX')
                    print('Y_', Y_.get_shape())

                with tf.name_scope('Loss'):
                    #cost = -1 * tf.log(Y_ + 0.0001) # Adding a small delta because log is not a continuous function. 
                    cost = -1 * energy + tf.log(z)
                    cost = tf.print(cost,[cost], "******** Cost(always postive)")
                    loss = tf.reduce_mean(cost) #get_cost(U, Z, Q) # 1- Y_[0]
                    loss = tf.print(loss, [loss], "**************** Loss")
                    tf.summary.scalar('loss', loss)

        return mu, sd, rbf_out, loss

    def get_spatial_loss(self):
        
        with self.graph.as_default():
            
            self.x_hat = self.get_decoded()
            spatial_loss = tf.losses.mean_squared_error(labels=self.x_, predictions=self.x_hat)
                
        return spatial_loss
    
    def get_temporal_loss(self):
        
        with self.graph.as_default():
            with tf.name_scope('Temporal_Autoencoder'):
                
                lstm_out = self.lstm(self.latent)
                print("LSTM: Images will be grouped (e.g. 10 images to 1 clip) into video clips when fed to LSTM",
                      lstm_out.get_shape())
                tf.summary.histogram("LSTM", lstm_out)

                self.mean, self.sigma, self.likelihood, likelihood_loss = self.rbf(lstm_out)
                print("Likelihood: ", self.likelihood.get_shape())
                tf.summary.scalar("Temporal_Loss", likelihood_loss)
                
        return likelihood_loss
    
    
    def get_optimizer_loss(self):
        
        with self.graph.as_default():
            self.x_hat = self.get_decoded()

            with tf.name_scope('Optimization'):
                
                spatial_loss = self.get_spatial_loss()
                temporal_loss = self.get_temporal_loss()
                
                loss = spatial_loss + temporal_loss
                tf.summary.scalar("loss",loss)
                
                starter_learning_rate = 0.001
                global_step = tf.Variable(0, trainable=False)
                #global_step = tf.print(global_step, [global_step], "########## Global Step Completed #############")
                learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                                   int(Params.epochs / 3), 0.5, staircase=True)
                train = tf.train.AdamOptimizer(learning_rate).minimize(loss, global_step=global_step)
                tf.summary.scalar("learning_rate",learning_rate)

            merged = tf.summary.merge_all()
        return train, merged, loss, self.likelihood, self.mean, self.sigma
        
        

In [7]:
loss_arr=[]
likelihood_arr=[]
network = Network()

m_b_s = 3*Params.frames_for_anomaly
with tf.Session(graph=network.graph) as sess:

    
    optim_loss = network.get_optimizer_loss()
    
    train_writer = tf.summary.FileWriter('logdir/cae_train', sess.graph)
    
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(Params.epochs):
        m_b_s_old = 0
        for m_b_s_new in range(m_b_s, Dataset.train_images_.shape[0], m_b_s  ):
            _, merged, loss, likelihood, mean, sigma = sess.run(optim_loss, 
                                                   feed_dict={network.x:Dataset.train_images_[m_b_s_old:m_b_s_new,...],
                                                            network.x_: Dataset.train_images[m_b_s_old:m_b_s_new,...]})

            m_b_s_old = m_b_s_new

        if epoch % 1 == 0:
            print('epoch:', epoch, 'loss:', loss, 'Likelihood:', np.asarray(likelihood).flatten(), 'Mean:', np.asarray(mean).flatten())
            train_writer.add_summary(merged, epoch)
            loss_arr.append(loss)
            likelihood_arr.append(np.mean(likelihood))
        
    
    #Dataset.latent = sess.run(network.latent,feed_dict={network.x:Dataset.train_images_,
    #                                        network.x_: Dataset.train_images})
    
    
    #print('Getting predictions for Training Data')
    #Dataset.reproduced_images_training = sess.run(network.x_hat,feed_dict={network.x:Dataset.train_images_[:30,... ],
    #                                        network.x_: Dataset.train_images[:30,... ]})
    #Dataset.likelihood_training, Dataset.mean_training, Dataset.sigma_training = sess.run([network.likelihood, network.mean, network.sigma],
    #                                                                                      feed_dict={network.x:Dataset.train_images_[:30,... ],
    #                                                                                    network.x_: Dataset.train_images[:30,... ]})
    
    #print('Getting predictions for Testing Data')
    #Dataset.reproduced_images = sess.run(network.x_hat,feed_dict={network.x:Dataset.test_images_[:30,... ],
    #                                        network.x_: Dataset.test_images[:30,... ]})
    #Dataset.likelihood, Dataset.mean, Dataset.sigma = sess.run([network.likelihood, network.mean, network.sigma],
    #                                                           feed_dict={network.x:Dataset.test_images_[:30,... ],
    #                                                            network.x_: Dataset.test_images[:30,... ]})
    
    

After 1st Conv (?, 154, 234, 32)
After 1st Pooling (?, 150, 230, 32)
After 2nd Conv (?, 146, 226, 16)
After 2nd Pooling (?, 142, 222, 16)
After 3rd Conv (?, 138, 218, 8)
After 3rd Pooling (Final Encoded) (?, 138, 218, 8)
Latent (?, 300)
After 1st conv transpose (?, 138, 218, 8)
After 2nd conv transpose (?, 142, 222, 16)
After 3rd conv transpose (?, 146, 226, 32)
After 4th conv transpose (?, 150, 230, 32)
After 5th conv transpose (?, 154, 234, 3)
After 6th conv transpose (final decoded) (?, 158, 238, 3)
After 1st Conv (?, 154, 234, 32)
After 1st Pooling (?, 150, 230, 32)
After 2nd Conv (?, 146, 226, 16)
After 2nd Pooling (?, 142, 222, 16)
After 3rd Conv (?, 138, 218, 8)
After 3rd Pooling (Final Encoded) (?, 138, 218, 8)
Latent (?, 300)
After 1st conv transpose (?, 138, 218, 8)
After 2nd conv transpose (?, 142, 222, 16)
After 3rd conv transpose (?, 146, 226, 32)
After 4th conv transpose (?, 150, 230, 32)
After 5th conv transpose (?, 154, 234, 3)
After 6th conv transpose (final decoded) (

ResourceExhaustedError: OOM when allocating tensor of shape [229408,300] and type float
	 [[Node: dense_1/kernel/Adam/Initializer/zeros = Const[dtype=DT_FLOAT, value=Tensor<type: float shape: [229408,300] values: [0 0 0]...>, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]

Caused by op 'dense_1/kernel/Adam/Initializer/zeros', defined at:
  File "/home/kvpcloud/.conda/envs/p3_gpu/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/kvpcloud/.conda/envs/p3_gpu/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/kvpcloud/.conda/envs/p3_gpu/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/kvpcloud/.conda/envs/p3_gpu/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/kvpcloud/.conda/envs/p3_gpu/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 486, in st

In [ ]:
plt.plot(loss_arr, label='Loss')
plt.plot(likelihood_arr, label='Likelihood')
plt.legend()

In [ ]:
#np.save('{}/latent_train'.format(Dataset.main),Dataset.latent)

In [ ]:
fig, axs = plt.subplots(6,5, figsize=(25,8))
axs = axs.ravel()

for i, imgs in enumerate(Dataset.test_images_[:30]):
    axs[i].imshow(imgs[:,:,:])

In [ ]:
fig, axs = plt.subplots(6,5, figsize=(25,8))
axs = axs.ravel()

for i, imgs in enumerate(Dataset.reproduced_images[:30]):
    axs[i].imshow(imgs[:,:,:])

In [ ]:
Dataset.likelihood, Dataset.mean, Dataset.sigma

In [ ]:
fig, axs = plt.subplots(6,5, figsize=(25,8))
axs = axs.ravel()

for i, imgs in enumerate(Dataset.train_images_[:30]):
    axs[i].imshow(imgs[:,:,:])

In [ ]:
fig, axs = plt.subplots(6,5, figsize=(25,8))
axs = axs.ravel()

for i, imgs in enumerate(Dataset.reproduced_images_training[:30]):
    axs[i].imshow(imgs[:,:,:])

In [ ]:
Dataset.likelihood_training, Dataset.mean_training, Dataset.sigma_training